In [35]:
pred_items = torch.load('../data/lightgcn_result.pt').flatten().numpy()

In [36]:
user_index = torch.repeat_interleave(torch.arange(pred_items.shape[0]//50), 50).numpy()

In [37]:
valUser, valItem = [], []
testUser, testItem = [], []
test_file = '/home/yamanishi/project/furusato_recommend/data/cf/test22_1_10.txt'
val_items_dict = defaultdict(list)
with open(test_file) as f:
    for l in tqdm(f.readlines()):
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            items = [int(i) for i in l[1:]]
            val_len =  int(len(items)//3)
            test_len = len(items)-val_len
            val_items = items[:val_len]
            test_items = items[val_len:]
            uid = int(l[0])
            valUser.extend([uid]*val_len)
            testUser.extend([uid]*test_len)
            valItem.extend(val_items)
            testItem.extend(test_items)
            val_items_dict[uid] = val_items

  0%|          | 0/154376 [00:00<?, ?it/s]

In [38]:
y = np.zeros_like(user_index)
for i,(u,p) in tqdm(enumerate(zip(user_index, pred_items))):
    if p in val_items_dict[u]:
        y[i] = 1

0it [00:00, ?it/s]

In [39]:
user_index = np.concatenate([user_index, np.array(valUser)])
pred_items = np.concatenate([pred_items, np.array(valItem)])
y = np.concatenate([y, np.ones(len(valItem))])

In [46]:
train_df = pd.DataFrame({'user': user_index,
      'item': pred_items,
      'buy': y})

In [44]:
def make_X(df):
    query_list = df['user'].value_counts()
    df = df.set_index(['user', 'item'])
    query_list = query_list.sort_index()
    df.sort_index(inplace=True)
    
    user_num_feature = np.load('/home/yamanishi/project/furusato_recommend/data/cb/user_numeric_feature22_1_10.npy')
    item_num_feature = np.load('/home/yamanishi/project/furusato_recommend/data/cb/product_numeric_feature22_1_10.npy')
    
    user_cat_feature = np.load('/home/yamanishi/project/furusato_recommend/data/cb/user_categorical_feature22_1_10.npy')
    item_cat_feature = np.load('/home/yamanishi/project/furusato_recommend/data/cb/item_categorical_feature22_1_10.npy')
    user_cat_X = user_cat_feature[df.index.get_level_values('user').values]
    item_cat_X = item_cat_feature[df.index.get_level_values('item').values]
    user_num_X = user_num_feature[:, :200][df.index.get_level_values('user').values]
    item_num_X = item_num_feature[:, :200][df.index.get_level_values('item').values]
    X= np.concatenate([user_num_X, user_cat_X, item_num_X, item_cat_X], axis=1)
    return df, X, query_list

In [47]:
train_df, train_X, train_query_list = make_X(train_df)

In [29]:
test_user_index = testUser
test_item_index = testItem
test_y = np.ones(len(testUser))
test_user_neg = np.arange(max(testUser)+1).repeat(10)
test_item_neg = np.random.randint(0, max(testItem), len(test_user_neg))
neg_y = np.zeros(len(test_user_neg))
test_user_index = np.concatenate([test_user_index, test_user_neg])
test_item_index = np.concatenate([test_item_index, test_item_neg])
test_y = np.concatenate([test_y, neg_y])

In [30]:
test_df = pd.DataFrame({'user': test_user_index,
      'item': test_item_index,
      'buy': test_y})

In [48]:
test_df, test_X, test_query_list = make_X(test_df)

In [49]:
lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "n_estimators":2000,
    "boosting_type": "gbdt",
    "num_leaves":31,
    "learning_rate":0.01,
    "importance_type": "gain",
    "random_state": 42,
}
lgb_fit_params = {
    "eval_metric":"ndcg",
    "eval_at":(1,2,3),
    "early_stopping_rounds": 50,
    "verbose": 10,
    "categorical_feature": categorical_cols,
}

NameError: name 'categorical_cols' is not defined

In [50]:
from typing import Optional
from collections import OrderedDict
from lightgbm.callback import CallbackEnv
from tqdm.auto import tqdm

class LgbmProgressBarCallback:
    description: Optional[str]
    pbar: tqdm

    def __init__(self, description: Optional[str] = None):
        self.description = description
        self.pbar = tqdm()

    def __call__(self, env: CallbackEnv):

        # 初回だけProgressBarを初期化する
        is_first_iteration: bool = env.iteration == env.begin_iteration

        if is_first_iteration:
            total: int = env.end_iteration - env.begin_iteration
            self.pbar.reset(total=total)
            self.pbar.set_description(self.description, refresh=False)

        # valid_setsの評価結果を更新
        if len(env.evaluation_result_list) > 0:
            # OrderedDictにしないと表示順がバラバラになって若干見にくい
            postfix = OrderedDict(
                [
                    (f"{entry[0]}:{entry[1]}", str(entry[2]))
                    for entry in env.evaluation_result_list
                ]
            )
            self.pbar.set_postfix(ordered_dict=postfix, refresh=False)

        # 進捗を1進める
        self.pbar.update(1)
        self.pbar.refresh()

In [52]:
import lightgbm as lgb

model = lgb.LGBMRanker(n_estimators=1000, random_state=0)
model.fit(
    X,
    y,
    group=query_list,
    callbacks=[
      LgbmProgressBarCallback(description="Model A"),
    ],
    eval_set=[(test_X, test_y)],
    eval_group=[list(test_query_list)]
)

0it [00:00, ?it/s]

In [34]:
customer_df = pd.read_csv('/home/yamanishi/project/furusato_recommend/data/cb/customer_cb22_1_10.csv')

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (33,58,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
def categorical_age(age):
    return min(age//5, 20)
def categorical_review_rate(review_rate):
    return review_rate//0.1

In [36]:
customer_df['categorical_age'] = customer_df['age'].apply(categorical_age)
customer_df['categorical_review_rate'] = customer_df['review_rate'].apply(categorical_review_rate)

In [37]:
cols = ['zip01', 'pref', 'categorical_age', 'mailmaga1_flg','sexuality','categorical_cnt', 'categorical_review_rate', 'categorical_review_cnt', 'yahoo_flg', 'credit_flg','category0', 'category1', 'category2', 'category3', 'category4']

In [38]:
from sklearn.preprocessing import LabelEncoder
customer_cat=customer_df[cols]
for col in cols:
    le = LabelEncoder()
    customer_cat[col] = le.fit_transform(customer_cat[col])

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [40]:
customer_cat = pd.concat([customer_cat, customer_df['age'], customer_df['review_cnt'], customer_df['cnt']], axis=1)

In [149]:
customer_categorical_feature = []
for col in cols:
    customer_categorical = pd.get_dummies(customer_df[col])
    customer_categorical_feature.append(customer_categorical)

In [150]:
customer_categorical_feature = np.concatenate(customer_categorical_feature, axis=1)

In [161]:
customer_categorical_feature.shape

(154376, 1370)

In [166]:
customer_categorical_feature = np.concatenate([customer_categorical_feature, np.nan_to_num(np.log10(customer_df['age'].values)).reshape(-1, 1), np.nan_to_num(np.log10(customer_df['cnt'])).reshape(-1, 1)], axis=1)

In [168]:
customer_categorical_feature.shape

(154376, 1372)

In [43]:
np.save('/home/yamanishi/project/furusato_recommend/data/cb/user_categorical_feature22_1_10.npy', customer_cat.values)

In [44]:
product_df = pd.read_csv('/home/yamanishi/project/furusato_recommend/data/cb/product_cb22_1_10.csv')

In [45]:
product_cols = ["categorical_price","categorical_review_cnt","categorical_review_rate","categorical_cnt", "categorical_minimum_donation_price",
"categorical_pref","categorical_town", 'category0', 'category1', 'category2', 'shipping_type', 'expensive_donation_flg', 'pickup_date']

In [48]:
from sklearn.preprocessing import LabelEncoder
product_cat=product_df[product_cols]
for col in product_cols:
    le = LabelEncoder()
    product_cat[col] = le.fit_transform(product_cat[col])

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [51]:
product_cat = pd.concat([product_cat, product_df['cost_price'], product_df['minimum_donation_price'], product_df['cnt'], product_df['review_cnt'], product_df['review_rate']], axis=1)

In [52]:
np.save('/home/yamanishi/project/furusato_recommend/data/cb/item_categorical_feature22_1_10.npy', product_cat.values)

In [189]:
product_categorical_feature = []
for col in cols:
    product_categorical = pd.get_dummies(product_df[col])
    product_categorical_feature.append(product_categorical)

In [190]:
product_categorical_feature = np.concatenate(product_categorical_feature, axis=1)

In [191]:
product_categorical_feature.shape

(28145, 1681)

In [194]:
product_df.columns[product_df.columns.str.contains('minimum')]

Index(['minimum_donation_price', 'categorical_minimum_donation_price'], dtype='object')

In [197]:
product_categorical_feature = np.concatenate([product_categorical_feature, np.nan_to_num(np.log10(product_df['cost_price'].values)).reshape(-1, 1), np.nan_to_num(np.log10(product_df['minimum_donation_price'])).reshape(-1, 1), np.nan_to_num(np.log10(product_df['cnt'])).reshape(-1, 1)], axis=1)

/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


In [198]:
np.save('/home/yamanishi/project/furusato_recommend/data/cb/item_categorical_feature22_1_10.npy',product_categorical_feature)

In [2]:
class TimeEncode(torch.nn.Module):
    def __init__(self, expand_dim, factor=5):
        super(TimeEncode, self).__init__()
        #init_len = np.array([1e8**(i/(time_dim-1)) for i in range(time_dim)])
        
        time_dim = expand_dim
        self.factor = factor
        self.basis_freq = torch.nn.Parameter((torch.from_numpy(1 / 10 ** np.linspace(0, 9, time_dim))).float())
        self.phase = torch.nn.Parameter(torch.zeros(time_dim).float())
        
        #self.dense = torch.nn.Linear(time_dim, expand_dim, bias=False)

        #torch.nn.init.xavier_normal_(self.dense.weight)
        
    def forward(self, ts):
        # ts: [N, L]
        batch_size = ts.size(0)
        seq_len = ts.size(1)
                
        ts = ts.view(batch_size, seq_len, 1)# [N, L, 1]
        map_ts = ts * self.basis_freq.view(1, 1, -1) # [N, L, time_dim]
        map_ts += self.phase.view(1, 1, -1)
        
        harmonic = torch.cos(map_ts)

        return harmonic

In [6]:
te = TimeEncode(64)
te(torch.randint(low=0, high=365, size=(128, 1)))

tensor([[[-0.9410,  0.9551,  0.3077,  ...,  1.0000,  1.0000,  1.0000]],

        [[-0.8999,  0.6689, -0.9170,  ...,  1.0000,  1.0000,  1.0000]],

        [[ 0.0398, -0.5338,  0.5307,  ...,  1.0000,  1.0000,  1.0000]],

        ...,

        [[-0.9183,  0.9029, -0.6799,  ...,  1.0000,  1.0000,  1.0000]],

        [[-0.9440,  0.1253,  0.7175,  ...,  1.0000,  1.0000,  1.0000]],

        [[-0.4794, -0.7640, -0.4638,  ...,  1.0000,  1.0000,  1.0000]]],
       grad_fn=<CosBackward0>)

In [7]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix

l = [[0, 10, 20],
     [30, 0, 40],
     [0, 0, 50]]
csc = csc_matrix(l)

In [13]:
data = [10, 20, 30, 40, 50]
row = [0, 0, 1, 1, 2]
col = [1, 2, 0, 2, 2]

In [16]:
csc2 = csr_matrix((data, (row, col)), shape=(3,3))

In [73]:
csc2.col

AttributeError: col not found

In [52]:
np.ravel(csc2[np.array([1, 0]),np.array([0, 1])])

array([30, 10])

In [54]:
transaction = pd.read_csv('/home/yamanishi/project/furusato_recommend/data/transaction22_12_10.csv')

In [55]:
transaction['donation_date'] = pd.to_datetime(transaction['donation_date'])

In [56]:
transaction['timestamp'] = transaction['donation_date']-transaction['donation_date'].min()

In [57]:
from datetime import timedelta
transaction['timestamp'] = transaction['timestamp']/timedelta(days=1)

In [61]:
a = torch.tensor([[4,5,6,7,8,9]])

b = torch.tensor([[0,1,2],
                  [3,2,1]])
a[b.flatten()].resshape(b.size())

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [64]:
b.flatten().size()

torch.Size([6])

In [68]:
a[b.reshape(-1)]

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [58]:
buy_timestamp = csr_matrix((transaction['timestamp'].values, (transaction['cf_user'].values, transaction['cf_product'].values)), size=(transaction['cf_user'].max()+1, transaction['cf_product'].max()))

In [76]:
n_user = transaction['cf_user'].max()+1

In [79]:
buy_timestamp[3046, 5021-n_user]

0.0

In [80]:
buy_timestamp.flags.writable

AttributeError: flags not found

In [59]:
with open('../data/cf/buy_timestamp22_12_10.pkl', 'wb') as f:
    pickle.dump(buy_timestamp, f)

In [81]:
with open('../data/cf/buy_timestamp22_12_10.pkl', 'rb') as f:
    bt=pickle.load(f)

In [83]:
bt[3892, 726]

0.0

In [39]:
transaction['cf_user']

0               0
1               0
2               0
3               1
4               2
            ...  
2637358    115916
2637359     43128
2637360     43128
2637361     35713
2637362     70716
Name: cf_user, Length: 2637363, dtype: int64

In [38]:
transaction['timestamp']

0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
           ...  
2637358    410.0
2637359    410.0
2637360    410.0
2637361    410.0
2637362    410.0
Name: timestamp, Length: 2637363, dtype: float64

In [11]:
csc[[1, 0],[0, 1]]

matrix([[30, 10]])

In [9]:
csc[[(1,0), (0,1)]]

IndexError: index results in >2 dimensions

In [84]:
import torch.nn as nn
class ScaledDotProductAttention(torch.nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = torch.nn.Dropout(attn_dropout)
        self.softmax = torch.nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):

        attn = torch.bmm(q, k.transpose(1, 2))
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -1e10)

        attn = self.softmax(attn) # [n * b, l_q, l_k]
        attn = self.dropout(attn) # [n * b, l_v, d]
                
        output = torch.bmm(attn, v)
        
        return output, attn

class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias=False)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5), attn_dropout=dropout)
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv

        mask = mask.repeat(n_head, 1, 1) # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)

        output = output.view(n_head, sz_b, len_q, d_v)
        
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)
        #output = self.layer_norm(output)
        
        return output, attn

In [85]:
mha = MultiHeadAttention(8, 128, 16, 16)

In [86]:
mha(torch.rand(128, 16), torch.rand(128, 16), torch.rand(128, 16))

ValueError: not enough values to unpack (expected 3, got 2)

In [1]:
a = torch.rand(3, 8)
b = torch.rand(3, 8)
torch.mean([a,b], dim=1).shape

TypeError: mean() received an invalid combination of arguments - got (list, dim=int), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [4]:
torch.cat([torch.rand(3,5), torch.rand(5,5)], dim=0).size()

torch.Size([8, 5])

In [1]:
favorite = pd.read_csv('/home/yamanishi/project/furusato_recommend/data/customer_favorite.csv', encoding='utf-16')

In [13]:
import re
def date_process(date):
    g=re.match(r'(\d+/\d+/\d+)', date)
    return g.group(1)

In [14]:
favorite.loc[0, 'create_date']

'10/8/2016 1:54:20 PM'

In [15]:
date_process(favorite.loc[0, 'create_date'])

'10/8/2016'

In [19]:
from tqdm import tqdm
tqdm.pandas()
favorite['date'] = favorite['create_date'].progress_apply(date_process) 

100%|██████████| 24297639/24297639 [00:32<00:00, 749956.46it/s]


In [20]:
favorite['date'] = pd.to_datetime(favorite['date'])

In [2]:
np.load('/home/yamanishi/project/furusato_recommend/data/cb/user_categorical_feature22_1_10.npy')

array([[925.        ,   3.        ,   7.        , ...,  37.        ,
          1.        ,  40.00098827],
       [297.        ,  10.        ,  10.        , ...,  54.        ,
          0.        ,  13.00009044],
       [217.        ,  13.        ,  10.        , ...,  52.        ,
          0.        ,  23.00034674],
       ...,
       [135.        ,  12.        ,  11.        , ...,  58.        ,
          0.        ,  12.00042311],
       [408.        ,  22.        ,  15.        , ...,  77.        ,
          0.        ,  10.00039217],
       [748.        ,  37.        ,  11.        , ...,  58.        ,
          0.        ,  10.99997505]])

In [5]:
np.load('/home/yamanishi/project/furusato_recommend/data/cb/item_categorical_feature22_1_10.npy').astype(int)

array([[ 9, 15, 12, ..., 13,  7,  4],
       [ 9, 19,  8, ..., 48, 63,  4],
       [ 9, 19, 10, ..., 65, 60,  4],
       ...,
       [ 0,  2,  2, ..., 10,  0,  0],
       [ 0,  4,  2, ..., 17,  0,  0],
       [ 0,  0,  2, ..., 16,  0,  0]])